### Connect to Foundry project

In [ ]:
# Import packages
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv(override=True) # Load environment variables from .env file

# Use the connection string to connect to your Foundry project
project_connection_string = os.getenv("AIPROJECT_CONNECTION_STRING")
try:
    project = AIProjectClient.from_connection_string(
        conn_str=project_connection_string, credential=DefaultAzureCredential()
    )
except Exception as e:
    print(f"Failed to connect to project: {e}")
    raise



### Create a simple agent

In [ ]:
## need to login using the command azd auth login
agent = project.agents.create_agent(
        model=os.getenv("CHAT_MODEL"),
        name="SimpleChatAgent",
        instructions="You are helpful AI assistant.",
     
    )
print(f"Created agent, agent ID: {agent.id}")

### Create a thread for conversation presistance

In [ ]:
# Create a thread
thread = project.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

### Create new message payload

In [ ]:
    # Get messages from the thread
message = project.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Hey, can you tell a joke about teddy bears?",
    )
print(f"Created message, ID: {message.id}")


### Run the thread

In [ ]:
run = project.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    # [END create_and_process_run]
print(f"Run finished with status: {run.status}")

if run.status == "failed":
        print(f"Run failed: {run.last_error}")

### Get the response

In [ ]:
messages = project.agents.list_messages(thread_id=thread.id)

for msg in messages.data:
    print(f"Message ID: {msg.id}")
    print(f"Role: {msg.role}")
    print("Content:")
    for content in msg.content:
        if content['type'] == 'text':
            print(content['text']['value'])
    print("-" * 50)

# Delete the agent once done
#project.agents.delete_agent(agent.id)
#print("Deleted agent")